<a href="https://colab.research.google.com/github/sakethsridhara/Milwaukee_bearings/blob/main/latentSpaceBearingsPlotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bearing Selection through latent space

## 1. Initialize

### Clone repo

In [1]:
# run this first time to clone the directory
!git clone https://github.com/sakethsridhara/Milwaukee_bearings.git
from os import chdir # to alter your working directory
chdir('/content/Milwaukee_bearings') # setting the working directory


Cloning into 'Milwaukee_bearings'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 91 (delta 42), reused 63 (delta 21), pack-reused 0
Receiving objects: 100% (91/91), 2.59 MiB | 3.49 MiB/s, done.
Resolving deltas: 100% (42/42), done.


### Python Imports

In [2]:
import numpy as np
import pandas as pd
import torch
import sys
import time
import matplotlib.pyplot as plt
import matplotlib
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon, Ellipse
import plotly.graph_objects as go
sys.path.insert(0,'/content/Milwaukee_bearings/src/')
from utilFuncs import to_np, to_torch
from materialEncoder import MaterialEncoder
from smallestEllipse import *
from plotly import __version__
import seaborn as sns
import scipy as sp
import copy


## 2. Read database from XLSX

In [3]:
def preprocessData():
  df = pd.read_excel('./data/bearingData.xlsx')
  dataIdentifier = {'classID':df[df.columns[0]],'name': df[df.columns[1]]} # name of the material and type
  trainInfo = np.log10(df[df.columns[2:]].to_numpy())
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  trainingData = normalizedData.clone().float()
  dataInfo = {'ID':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'OD':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'Width':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'DynamicLoadRating':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]},\
              'StaticLoadRating':{'idx':4,'scaleMin':dataScaleMin[4], 'scaleMax':dataScaleMax[4]},\
              'RPM':{'idx':5,'scaleMin':dataScaleMin[5], 'scaleMax':dataScaleMax[5]},\
              'Cost':{'idx':6,'scaleMin':dataScaleMin[6], 'scaleMax':dataScaleMax[6]}}
  return trainingData, dataInfo, dataIdentifier, trainInfo
trainingData, dataInfo, dataIdentifier, trainInfo = preprocessData()
numMaterialsInTrainingData, numFeatures = trainingData.shape

## 3. VAE Setup

In [4]:
latentDim = 2
hiddenDim = [250,250]
numEpochs = 30000
klFactor = 5e-5
learningRate = 2e-3
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}
materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)


In [5]:
savedNet = './data/vaeTrained.pt'
materialEncoder.vaeNet = torch.load(savedNet,map_location=torch.device('cpu'))

# start = time.perf_counter()
# convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate)
# print('training time : {:.2F} '.format(time.perf_counter() - start))

### Error estimation

In [6]:
matidxs = np.arange(trainInfo.shape[0]).astype(int)
props = ['ID','OD','Width','StaticLoadRating','DynamicLoadRating','RPM','Cost']

def unnormalize(val, minval ,maxval):
  return 10.**(minval + (maxval-minval)*val)
def decodeAll():
  vae = materialEncoder.vaeNet
  decoded = vae.decoder(vae.encoder.z)
  matProp = {'ID':None,'OD':None,'Width':None,'StaticLoadRating':None,'DynamicLoadRating': None, 'RPM':None,'Cost':None}

  for k in props:
    idx = materialEncoder.dataInfo[k]['idx']
    scaleMax = materialEncoder.dataInfo[k]['scaleMax']
    scaleMin = materialEncoder.dataInfo[k]['scaleMin']
    matProp[k] = unnormalize(decoded[:,idx], scaleMin ,scaleMax)#scaleMin + decoded[:,idx]*(scaleMax - scaleMin)
  return matProp

matProp = decodeAll()
merr = -1000000000.

maxError = {'ID':merr,'OD':merr,'Width':merr,'StaticLoadRating':merr,'DynamicLoadRating':merr, 'RPM':merr,'Cost':merr}
# print('\n \n \t \t ------RECON ERROR (%)----------')
# print('Catalog name', end = '\t')
errList = torch.zeros(trainInfo.shape[0],7);
for p in props:
    print(f"\t{p}", end = '\t')
for i in range(trainInfo.shape[0]):
  count = 0;

  # if(i in matidxs): #
    # print(f"\n  {dataIdentifier['name'][i]} ", end = '')

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    trueData = 10**trainInfo[i,idx]
    reconData = matProp[p][i]
    err = torch.abs(100.*(trueData - reconData)/trueData).to('cpu')
    errList[i,count] = err
    count = count + 1;
    if(err > maxError[p]):
      maxError[p] = err
    # if(i in matidxs):
      # print('\t {:.1F} \t'.format(err), end='')


print('\n Max Error', end = '')
for p in props:
  print('\t {:.1F}'.format(maxError[p]), end='')

print("\n Mean Error:")
print(torch.mean(errList,0))

	ID		OD		Width		StaticLoadRating		DynamicLoadRating		RPM		Cost	
 Max Error	 2.7	 3.2	 3.2	 7.5	 10.5	 2.7	 5.0
 Mean Error:
tensor([0.6506, 0.8017, 0.6719, 1.4935, 1.6631, 0.6763, 1.0909],
       grad_fn=<MeanBackward1>)


## 4. Latent Space Representation


In [40]:
# @title
def plotlyLatent():
    clrs = ['blue', 'green']
    mrkrSet = [0,2]
    bearingClass = ['Ball Bearing','Roller Bearing']
    colorcol = dataIdentifier['classID']
    ptLabel = dataIdentifier['name']
    autoencoder = materialEncoder.vaeNet
    z = autoencoder.encoder.z.to('cpu').detach().numpy()
    scatter_traces = []
    for i in range(np.max(colorcol)+1):
      zMat = np.vstack((z[colorcol == i,0], z[colorcol == i,1])).T
      txt = ptLabel[colorcol == i]
      scatter_trace = go.Scatter(
                                  x=zMat[:, 0],
                                  y=zMat[:, 1],
                                  mode='markers',
                                  text = txt,
                                  name = bearingClass[i],
                                  hoverinfo= 'x+y+text',
                                  marker=dict(size=8,
                                              color=clrs[i],
                                              colorscale='Viridis',
                                              opacity=0.7,
                                              symbol = mrkrSet[i]))
      scatter_traces.append(scatter_trace)
      hull = ConvexHull(zMat)
      cent = np.mean(zMat, 0)
      pts = []
      for pt in zMat[hull.simplices]:
          pts.append(pt[0].tolist())
          pts.append(pt[1].tolist())

      pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                      p[0] - cent[0]))
      pts = pts[0::2]  # Deleting duplicates
      enclosing_ellipse = welzl(np.array(pts, dtype=float))
      # plot resulting ellipse
      center,a,b,t = enclosing_ellipse

      elli = sample_ellipse(enclosing_ellipse,1000)
      scatter_trace = go.Scatter(x=elli[:, 0],
          y=elli[:, 1],
          mode= 'lines',
          text = bearingClass[i],
          hoverinfo ='skip',
          showlegend= False,
          marker=dict(size=8,
              color=clrs[i],
              colorscale='Viridis',
              opacity=0.7))
      scatter_traces.append(scatter_trace)



    scatter_plotly = go.Figure(data=scatter_traces)
    scatter_plotly.update_layout(plot_bgcolor='white',showlegend=True)
    scatter_plotly.update_layout(title='Latent Space of Bearings',title_x=0.5)

    scatter_plotly.update_layout(xaxis_range=[-3, 3],  # Set the x-axis limits
        yaxis_range=[-3, 3], width = 800, height = 800);
    scatter_plotly.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    scatter_plotly.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    scatter_plotly.update_layout(xaxis=dict(zerolinecolor='lightgrey'), yaxis=dict(zerolinecolor='lightgrey'))
    scatter_plotly.show(renderer='colab')
    return scatter_plotly,scatter_traces



Visual representation

In [41]:
scatter_plotly, scatter_traces = plotlyLatent()

In [44]:
def plotlyLatentContours(props = ['RPM'],cutOff = [0,100]):
  n = 100
  zmin, zmax = -3,3
  xi = np.linspace(zmin, zmax, n)
  yi = np.linspace(zmin, zmax, n)
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim'])).to('cpu')
  Z[:,0], Z[:,1] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet.to('cpu')
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)



  #-------------------------------------------#

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = to_np(10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin)))
    levs = np.logspace(np.log10(min(matPropVal))*0.5, np.log10(max(matPropVal)), 40)
    contour_trace = go.Contour(z= matPropVal.reshape((n,n)), x=xi, y=yi,
        colorscale='Viridis',
        showscale=False,  # Hide the color scale
        contours=dict(start = cutOff[0], end = cutOff[1], coloring='lines', showlabels=True),
        colorbar=dict(len=0.6),
        hoverinfo='z'
    )


    return contour_trace



In [52]:
prop = ['RPM']
cutOff = [30000,70000]
# props = ['ID','OD','Width','StaticLoadRating','DynamicLoadRating','RPM','Cost']


scatter_contour = copy.deepcopy(scatter_plotly)

contour_trace = plotlyLatentContours(prop,cutOff)
scatter_contour.add_trace(contour_trace)
scatter_contour.update_layout(title=(f'{prop}'))


scatter_contour.show(renderer='colab')



In [51]:
prop = ['RPM']
cutOff = [30000,40000]

# props = ['ID','OD','Width','StaticLoadRating','DynamicLoadRating','RPM','Cost']


scatter_contour = copy.deepcopy(scatter_plotly)

contour_trace = plotlyLatentContours(prop,cutOff)
scatter_contour.add_trace(contour_trace)

prop = ['ID']
cutOff = [0.25,1]

contour_trace = plotlyLatentContours(prop,cutOff)
scatter_contour.add_trace(contour_trace)


scatter_contour.update_layout(title=(f'{prop}'))


scatter_contour.show(renderer='colab')

In [73]:
def plotlyFilledContours(props = ['RPM'],cutOff = [10000,50000]):
  n = 100
  zmin, zmax = -3,3
  xi = np.linspace(zmin, zmax, n)
  yi = np.linspace(zmin, zmax, n)
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim'])).to('cpu')
  Z[:,0], Z[:,1] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet.to('cpu')
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)



  #-------------------------------------------#

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = to_np(10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin)))
    levs = np.logspace(np.log10(min(matPropVal))*0.5, np.log10(max(matPropVal)), 40)
    contour_trace = go.Contour(z= matPropVal.reshape((n,n)), x=xi, y=yi,
        colorscale='Viridis',
        showscale=False,  # Hide the color scale
                               zmin = cutOff[0],zmax = cutOff[1],
        colorbar=dict(len=0.6),
        hoverinfo='z',
            line=dict(width=0),  # set line width to 0 for filled contour
    )


    return contour_trace



In [74]:
prop = ['RPM']
cutOff = [30000,40000]
# props = ['ID','OD','Width','StaticLoadRating','DynamicLoadRating','RPM','Cost']


scatter_contour = copy.deepcopy(scatter_plotly)

contour_trace = plotlyFilledContours(prop,cutOff)
scatter_contour.add_trace(contour_trace)
scatter_contour.update_layout(title=(f'{prop}'))


scatter_contour.show(renderer='colab')